In [ ]:
pip install happytransformer

In [2]:
from datasets import load_dataset

In [3]:
from happytransformer import HappyTextToText
model = HappyTextToText("T5", "t5-base")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

02/04/2022 11:07:40 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [4]:
train_data = load_dataset("jfleg", split='validation[:]')
eval_data = load_dataset("jfleg", split='test[:]')

Downloading:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/27.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/26.6k [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset jfleg downloaded and prepared to /root/.cache/huggingface/datasets/jfleg/default/1.0.0/ed4ab2367351fe31949f48849ae6732b164f0d5ea6bb5d4357ff4293ac89511b. Subsequent calls will reuse this data.


In [5]:
for i in train_data["corrections"][:2]:
  print(i)
  print(i[0])

['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']
So I think we would not be alive if our ancestors did not develop sciences and technologies . 
['Not for use with a car . ', 'Do not use in the car . ', 'Car not for use . ', 'Can not use the car . ']
Not for use with a car . 


In this below step we are adding "grammar:" as prefix , Because the model we chose is capable of doing more than one task.

In [6]:
import csv
def get_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset: 
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]: 
                if input_text and correction:
                    writter.writerow([input_text, correction])
get_csv("train.csv", train_data)
get_csv("eval.csv", eval_data)

In [7]:
from happytransformer import TTTrainArgs
args = TTTrainArgs(batch_size=8)
model.train("train.csv", args=args)

02/04/2022 11:53:06 - INFO - happytransformer.happy_transformer -   Preprocessing training data...


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-62ab5927d4354d57/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

02/04/2022 11:53:14 - INFO - happytransformer.happy_transformer -   Training...
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3016
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1131


Step,Training Loss
500,0.580800
1000,0.437300




Training completed. Do not forget to share your model on huggingface.co/models =)




In [8]:
loss = model.eval("eval.csv")
print("loss: ", loss.loss)

02/04/2022 15:21:15 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5a429e745a46ab5b/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 2988
  Batch size = 1


loss:  0.44850626587867737


We have to add "grammar:" as prefix while predicting too.

In [9]:
from happytransformer import TTSettings
settings =  TTSettings(num_beams=5, min_length=1, max_length=100)
result = model.generate_text("grammar: This sentences, has bads grammar and spelling!", args=settings)
print(result.text)

This sentence has bad grammar and spelling!


In [15]:
model.generate_text("grammar: I am enjoys, writtings article ons AI.", args=settings).text

'I am enjoying writing an article on AI.'

In [18]:
model.save("grammar-correction-model")

Configuration saved in grammar-correction-model/config.json
Model weights saved in grammar-correction-model/pytorch_model.bin
tokenizer config file saved in grammar-correction-model/tokenizer_config.json
Special tokens file saved in grammar-correction-model/special_tokens_map.json
Copy vocab file to grammar-correction-model/spiece.model
